In [34]:
import connections_config
import pandas as pd
import uuid

In [35]:
#Guardar conexion a la base de datos de akademos en la variable "akademos"
akademos = connections_config.akademos_data_base

#Guardar conexion al Mercado de Datos en la variable "datamart"
datamart =connections_config.datamart_data_base

In [36]:
#Consulta para extracción de datos
query = """SELECT tc.id AS id_motivo,te.fecha_creacion AS fecha, extract(YEAR from te.fecha_creacion) AS anno, extract(month from te.fecha_creacion) AS mes, EXTRACT (DAY FROM te.fecha_creacion) AS dia, te.tramite_anterior AS tramite_anterior
	        FROM tramite_tbd_tramite_estudiante AS te
	            INNER JOIN tramite_tbr_tramite_configuracion AS tc
    		        ON te.tipo_tramite_estudiante = tc.id"""

query_almacen = """SELECT * FROM dim_motivo"""

In [37]:
#Cargar los movimienitos de la BD en un DataFrame
df = pd.read_sql(query, akademos)

#cargar los movimientos del Mercado en un DataFrame
df_datamart = pd.read_sql(query_almacen, datamart)
df

,id_motivo,fecha,anno,mes,dia,tramite_anterior
0,d0d9d3da-f51c-4baa-91e3-a6679068da5f,2021-10-20 02:22:13.155492+02:00,2021.0,10.0,20.0,None
1,33ddf217-f253-480f-a7db-b70c643fa663,2024-07-15 02:22:13.155492+02:00,2024.0,7.0,15.0,None
2,5ac77cc3-3e54-43b3-a989-a509d216226c,2022-11-21 01:22:13.155492+01:00,2022.0,11.0,21.0,None


In [38]:
df['id_tiempo'] = df.apply(lambda row: int(row['anno']*10000 + row['mes']*100 + row['dia']), axis = 1)
df

,id_motivo,fecha,anno,mes,dia,tramite_anterior,id_tiempo
0,d0d9d3da-f51c-4baa-91e3-a6679068da5f,2021-10-20 02:22:13.155492+02:00,2021.0,10.0,20.0,None,20211020
1,33ddf217-f253-480f-a7db-b70c643fa663,2024-07-15 02:22:13.155492+02:00,2024.0,7.0,15.0,None,20240715
2,5ac77cc3-3e54-43b3-a989-a509d216226c,2022-11-21 01:22:13.155492+01:00,2022.0,11.0,21.0,None,20221121


In [42]:
#df_bajas = pd.DataFrame()
#df_bajas = df[df['id_motivo'] == '']

#valores_a_buscar = [uuid.UUID('d0d9d3da-f51c-4baa-91e3-a6679068da5f'), uuid.UUID('7bdc04be-f760-4cae-a4e2-33f34b47c076')]

#df_altas = pd.DataFrame()
df_altas = df[df['id_motivo'] == 'd0d9d3da-f51c-4baa-91e3-a6679068da5f']

#df_otras_bajas = pd.DataFrame()
#df_graduados_egresados = pd.DataFrame()
df_altas

,id_motivo,fecha,anno,mes,dia,tramite_anterior,id_tiempo


In [ ]:
#Compara los DataFrame y quedarse solo con los valores nuevos, los que no se repiten (En caso de annadir nuevos motivos el sistema)
df_resultado = df[~df['id_motivo'].isin(df_datamart['id_motivo'])]
df_resultado

,id_motivo,fecha,anno,mes,dia,tramite_anterior,id_tiempo


In [ ]:
#Guardar motivos en el Mercado
df_resultado.to_sql("dim_motivo", con = datamart, if_exists = "append", index = False)

0